In [1]:
%load_ext autoreload
%autoreload 2

from fastai.text.all import *
from fastai.vision.all import *
import pandas as pd
import torch
from tqdm.notebook import tqdm

from utils import get_dls

In [2]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

# # tensorflow RNG
# tf.random.set_seed(seed)

In [3]:
path = Path("/mnt/nas/backups/08-07-2020/desktopg01/lisa/Data/small_flow")

In [4]:
dls = get_dls(path, 64, 224)

In [5]:
test_items = get_image_files(path, folders="test")

In [6]:
test_dl = dls.test_dl(test_items, with_labels=True)

In [7]:
class GetActs(HookCallback):
    def __init__(self, modules=None, remove_end=True, detach=True, cpu=True):
        super().__init__(modules, None, remove_end, True, detach, cpu)
        self.acts = L()
    def hook(self, m, i, o): return o
    def after_pred(self): self.acts += self.hooks.stored
    def before_fit(self):
        super().before_fit()
        self.acts = L()

In [8]:
learn = cnn_learner(dls, resnet50, loss_func=CrossEntropyLossFlat())

In [9]:
learn.add_cb(GetActs([learn.model[1][1]]))

In [10]:
learn.load("best_image_weights_224")

In [11]:
learn.validate()

(#1) [0.3971825838088989]

In [12]:
valid_preds = learn.get_acts.acts

In [16]:
valid_preds = torch.cat(list(valid_preds)); valid_preds.shape

torch.Size([102997, 4096])

In [18]:
assert len(valid_preds) == len(dls.valid_ds)

In [19]:
for idx, item in enumerate(dls.valid.items):
    filename = Path("./activations/img")/re.search(r'val\/[^\/]*\/[^.]*',item.as_posix())[0]
    filename.parent.mkdir(parents=True, exist_ok=True)
    torch.save(valid_preds[idx].clone(), filename.as_posix() + ".pt")
    print(f"Saving example {idx+1}", end='\r', flush=True)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
dls.train.shuffle = False

In [21]:
dls.train.get_idxs()[:5]

[0, 1, 2, 3, 4]

In [ ]:
learn.get_preds(0)

(TensorImage([[1.5496e-01, 9.4552e-03, 1.6290e-02, 6.1323e-01, 3.0069e-02, 1.7599e-01],
         [8.0321e-02, 6.5189e-03, 7.0756e-02, 7.5476e-01, 1.0702e-02, 7.6943e-02],
         [2.4597e-02, 9.5386e-03, 7.1015e-03, 8.7932e-01, 4.3612e-02, 3.5828e-02],
         ...,
         [4.3153e-04, 1.7354e-04, 9.5573e-04, 7.0746e-02, 1.1550e-03, 9.2654e-01],
         [3.5124e-02, 5.4816e-02, 9.0227e-03, 7.9092e-01, 7.5193e-02, 3.4928e-02],
         [5.3208e-02, 7.5849e-03, 1.6632e-02, 5.4792e-01, 2.4147e-02, 3.5051e-01]]),
 TensorCategory([0, 0, 0,  ..., 5, 5, 5]))

In [ ]:
train_preds = learn.get_acts.acts

In [ ]:
train_preds = torch.cat(list(train_preds)); train_preds.shape

torch.Size([158308, 4096])

In [ ]:
assert len(train_preds) == len(dls.train_ds)

In [26]:
for idx, item in enumerate(dls.train.items):
    filename = Path("./activations/img")/re.search(r'train\/[^\/]*\/[^.]*',item.as_posix())[0]
    filename.parent.mkdir(parents=True, exist_ok=True)
    torch.save(train_preds[idx].clone(), filename.as_posix() + ".pt")
    print(f"Saving example {idx+1}", end='\r', flush=True)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [29]:
learn.get_preds(dl=test_dl)

(TensorImage([[0.0026, 0.0408, 0.0011, 0.8655, 0.0784, 0.0116],
         [0.0033, 0.0198, 0.0010, 0.9566, 0.0157, 0.0036],
         [0.0361, 0.0530, 0.0088, 0.8587, 0.0174, 0.0260],
         ...,
         [0.0252, 0.0195, 0.0184, 0.8575, 0.0319, 0.0474],
         [0.0036, 0.0109, 0.0010, 0.9016, 0.0717, 0.0112],
         [0.0480, 0.0090, 0.0418, 0.6294, 0.0117, 0.2600]]),
 TensorCategory([0, 0, 0,  ..., 5, 5, 5]))

In [30]:
test_preds = learn.get_acts.acts

In [31]:
test_preds = torch.cat(list(test_preds)); test_preds.shape

torch.Size([98577, 4096])

In [32]:
assert len(test_preds) == len(test_dl.dataset)

In [33]:
for idx, item in enumerate(test_dl.items):
    filename = Path("./activations/img")/re.search(r'test\/[^\/]*\/[^.]*',item.as_posix())[0]
    filename.parent.mkdir(parents=True, exist_ok=True)
    torch.save(test_preds[idx].clone(), filename.as_posix() + ".pt")
    print(f"Saving example {idx+1}", end='\r', flush=True)